In [1]:
import pandas as pd
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, coo_matrix
from sklearn.cluster import MiniBatchKMeans

In [2]:
data = []

with open('cosmo_content_storage_final_cut.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))

In [3]:
df = pd.DataFrame.from_records(data)
df

,url,doc_id,description,ts,title
0,https://vz.ru/news/2019/12/6/1012187.html,1000029981939875422,У России есть необходимые конкурентные преимущ...,1575625264,Новак заявил о способности России стать лидеро...
1,https://meduza.io/news/2019/12/06/sk-otchitals...,1000103932845397396,По делу об убийстве главы ингушского центра «Э...,1575634419,"СК отчитался о поимке банды, причастной к убий..."
2,http://www.vestikavkaza.ru/material/283355,1000115462666782749,Президент Азербайджана Ильхам Алиев и первый в...,1575447101,Ильхам Алиев и Мехрибан Алиева посетили выстав...
3,http://krizis-kopilka.ru/archives/70325,1000129039443894284,Проблемы и вызовы языковой политики анализируе...,1575767444,Язык и идеология
4,https://radiosputnik.ria.ru/20191203/156189583...,1000131983536261699,Отношения России и Турции не являются альтерна...,1575359865,Эрдоган: отношения Турции с РФ не альтернативн...
...,...,...,...,...,...
88363,https://lenta.ru/news/2019/12/05/money/,999910919457925598,Производитель электроники Xiaomi запустил прил...,1575559155,Xiaomi начала выдавать микрокредиты
88364,https://megaobzor.com/Xbox-One-otlichno-prodal...,999919369677233771,Многие пользователи специально ждут «Черную пя...,1575391338,Xbox One отлично продалась с Черную пятницу
88365,https://car.ru/news/automobili/59574-kseniyu-b...,999942425463634000,Ксению Бородину раздражают хамы на дорогах,1575543490,Ксению Бородину раздражают хамы на дорогах
88366,https://www.livesport.ru/football/2019/12/05/m...,99994363695107503,Главный тренер сборной Бельгии поделился мнени...,1575636048,Наставник сборной Бельгии считает Италию фавор...


In [4]:
tfidf = TfidfVectorizer()
description = tfidf.fit_transform(df["description"].fillna(""))
title = tfidf.fit_transform(df["title"].fillna(""))
url = tfidf.fit_transform(df["url"].fillna(""))

In [5]:
train = hstack([description, title, url, coo_matrix(df['ts'].values).T])

In [56]:
km = MiniBatchKMeans(n_clusters=3000, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000)

In [57]:
km.fit(train)

/home/gb/made/made/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=1000 should be larger than k=3000. Setting it to 3*k
  init_size=init_size)


MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
                init_size=1000, max_iter=100, max_no_improvement=10,
                n_clusters=3000, n_init=1, random_state=None,
                reassignment_ratio=0.01, tol=0.0, verbose=0)

In [58]:
answer = pd.DataFrame(columns=['doc_id', 'cat'])
answer['doc_id'] = df['doc_id']
answer['cat'] = km.predict(train)

In [59]:
cluster = pd.read_json('cluster_final_cut_train.json', typ='series')

In [60]:
cluster.index = cluster.index.astype(int)

In [61]:
answer.loc[~(answer['doc_id'].isin(cluster.index))].dropna(how='all').to_csv('sub.txt', header=True, index=False)

In [62]:
cluster

8436846726673829479     516
4596566925810077827    2054
5624584102393981667     328
267057939486240219     2133
3318162853060722078    2266
                       ... 
2441688416900780817    2690
4025509225831026607     856
5544787745191406164    3036
6430542681857951713     962
7498798123207436785     395
Length: 26510, dtype: int64

In [63]:
answer.loc[(answer['doc_id'].isin(cluster.index))].dropna(how='all')

,doc_id,cat
3,1000129039443894284,2547
4,1000131983536261699,714
7,1000369819866877640,24
8,1000376594041833717,1711
9,1000567670258318930,1705
...,...,...
88347,998388923536420373,2203
88353,998760668593746561,1654
88358,999406325296585876,2171
88362,999909502498232052,790


In [64]:
cluster = cluster.reset_index(0).rename(columns={'index' : 'doc_id'})

In [65]:
answer.merge(cluster, on=['doc_id'], how='left').groupby(['cat', 0])['doc_id'].apply(list)

cat   0     
0     38.0                           [3232290667042315565]
      42.0      [1345866204491910867, 4867128903165914464]
      166.0                          [6789625144217761353]
      167.0                          [9198679327863849365]
      205.0     [1974170443910103237, 4098887186882959297]
                                   ...                    
2998  2557.0                         [8200218442482348237]
      2789.0                          [950568778691917378]
2999  96.0                           [7510399335969511898]
      362.0                          [6979664996965258792]
      859.0                          [8277663763494336310]
Name: doc_id, Length: 24877, dtype: object